In [ ]:
# First install required packages:
# pip install ultralytics opencv-python pyserial

import cv2
from ultralytics import YOLO
import serial
import time
import threading
import numpy as np

# Load YOLOv8 model
model = YOLO("yolov8n.pt")
model.conf = 0.3  
model.iou = 0.5   

# Camera settings
CAMERA_WIDTH = 640
CAMERA_HEIGHT = 480

# Global variables
ped_count = 0
veh_count = 0
ped_frame_display = None
veh_frame_display = None
processing_active = True
detection_active = True

# Moving average for counts
PED_SMOOTHING_WINDOW = 5
VEH_SMOOTHING_WINDOW = 5
ped_count_history = []
veh_count_history = []

# Initialize serial communication (optional)
def initialize_serial():
    try:
        ser = serial.Serial('COM3', 9600, timeout=1)
        print("Connected to COM3 successfully")
        time.sleep(2)  
        return ser
    except serial.SerialException as e:
        print(f"Serial connection disabled: {e}")
        return None

# Single camera processing for both pedestrian and vehicle detection
def process_camera():
    global ped_count, veh_count, ped_frame_display, veh_frame_display, processing_active
    global ped_count_history, veh_count_history
    
    cap = cv2.VideoCapture(0)  # Use default webcam (index 0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, CAMERA_WIDTH)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, CAMERA_HEIGHT)
    
    frame_skip = 3  # Process every 3rd frame to reduce load
    frame_count = 0
    vehicle_classes = [2, 3, 5, 7]
    
    while processing_active and cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            time.sleep(0.1)
            continue
            
        frame_count += 1
        if frame_count % frame_skip != 0:
            continue
            
        processed_frame = frame.copy()
        ped_frame = frame.copy()
        veh_frame = frame.copy()
        
        if detection_active:
            try:
                # Run detection once for all relevant classes
                results = model(processed_frame, classes=[0, 2, 3, 5, 7], verbose=False)
                
                # Reset counts
                current_ped = 0
                current_veh = 0
                
                for result in results:
                    for box in result.boxes:
                        cls_id = int(box.cls[0])
                        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                        conf = float(box.conf[0])
                        
                        # Draw bounding boxes and count objects
                        if cls_id == 0:  # Pedestrian
                            current_ped += 1
                            cv2.rectangle(ped_frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
                            cv2.putText(ped_frame, f"Person: {conf:.2f}", 
                                        (int(x1), int(y1)-10), 
                                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                        elif cls_id in vehicle_classes:  # Vehicles
                            current_veh += 1
                            cv2.rectangle(veh_frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)
                            cv2.putText(veh_frame, f"{model.names[cls_id]}: {conf:.2f}", 
                                        (int(x1), int(y1)-10), 
                                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
                
                # Update moving averages
                ped_count_history.append(current_ped)
                veh_count_history.append(current_veh)
                
                if len(ped_count_history) > PED_SMOOTHING_WINDOW:
                    ped_count_history.pop(0)
                if len(veh_count_history) > VEH_SMOOTHING_WINDOW:
                    veh_count_history.pop(0)
                
                ped_count = int(sum(ped_count_history) / len(ped_count_history))
                veh_count = int(sum(veh_count_history) / len(veh_count_history))
                
            except Exception as e:
                print(f"Detection error: {e}")
        
        # Update display frames
        ped_frame_display = ped_frame
        veh_frame_display = veh_frame
    
    cap.release()

# Traffic light control (optional)
def control_traffic_lights(ser):
    global ped_count, veh_count, processing_active
    last_command = ""
    
    while processing_active:
        try:
            if ser is None:  # Skip if no serial connection
                time.sleep(1)
                continue
                
            command = ""
            if ped_count > 0 and (ped_count >= veh_count or veh_count == 0):
                command = f"PED_GREEN {ped_count * 10}"
            elif veh_count > 0:
                command = f"VEH_GREEN {veh_count * 20}"
            else:
                command = "VEH_GREEN 20"
            
            if command != last_command:
                ser.write(f"{command}\n".encode())
                print(f"Command: {command}")
                last_command = command
            
            time.sleep(max(ped_count * 10 if 'PED' in command else veh_count * 20, 1))
        except Exception as e:
            print(f"Traffic control error: {e}")
            time.sleep(1)

# Display function 
def display_feeds():
    global ped_frame_display, veh_frame_display, processing_active, detection_active
    cv2.namedWindow("Traffic Monitoring System", cv2.WINDOW_NORMAL)
    
    while processing_active:
        if ped_frame_display is not None and veh_frame_display is not None:
            try:
                # Resize frames to match height
                height = 480
                ped_display = cv2.resize(ped_frame_display, (640, height))
                veh_display = cv2.resize(veh_frame_display, (640, height))
                
                # Add overlays
                status = "DETECTION ON" if detection_active else "DETECTION OFF"
                cv2.rectangle(ped_display, (0, 0), (250, 60), (0, 0, 0), -1)
                cv2.putText(ped_display, f"Pedestrians: {ped_count}", (10, 30), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                cv2.putText(ped_display, status, (10, 55), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
                
                cv2.rectangle(veh_display, (0, 0), (250, 60), (0, 0, 0), -1)
                cv2.putText(veh_display, f"Vehicles: {veh_count}", (10, 30), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                cv2.putText(veh_display, "Press 'd' to toggle detection", (10, 55), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
                
                # Combine frames
                combined = np.hstack((ped_display, veh_display))
                cv2.imshow("Traffic Monitoring System", combined)
                
                key = cv2.waitKey(1) & 0xFF
                if key == ord('q'):
                    processing_active = False
                elif key == ord('d'):
                    detection_active = not detection_active
                    print(f"Detection {'enabled' if detection_active else 'disabled'}")
                    
            except Exception as e:
                print(f"Display error: {e}")
        time.sleep(0.03)

def main():
    global processing_active
    print("Starting Traffic Monitoring System")
    print("Press 'q' to quit, 'd' to toggle detection")
    
    ser = initialize_serial()  # This will be None if no serial connection
    
    threads = [
        threading.Thread(target=process_camera),
        threading.Thread(target=control_traffic_lights, args=(ser,)),
        threading.Thread(target=display_feeds)
    ]
    
    for thread in threads:
        thread.daemon = True
        thread.start()
    
    try:
        while processing_active:
            time.sleep(0.1)
    except KeyboardInterrupt:
        print("\nShutting down...")
    finally:
        processing_active = False
        for thread in threads:
            thread.join(timeout=1.0)
        if ser is not None:
            ser.close()
        cv2.destroyAllWindows()
        print("System shutdown complete")

if __name__ == "__main__":
    main()

Starting Traffic Monitoring System
Press 'q' to quit, 'd' to toggle detection
Serial connection disabled: could not open port 'COM3': FileNotFoundError(2, 'The system cannot find the file specified.', None, 2)
